In [16]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler

In [2]:
df = pd.read_csv('/Users/hrishityelchuri/Documents/windPred/raw/8.52 hrishit data.csv')

In [3]:
df['PeriodEnd'] = pd.to_datetime(df['PeriodEnd'])
df['PeriodStart'] = pd.to_datetime(df['PeriodStart'])

In [4]:
df = df.sort_values('PeriodEnd')

In [ ]:
def create_multivariate_lagged_dataset(df, target_col, feature_cols, lag=3):
    """
    Create supervised learning data from multivariate time series.
    
    Parameters:
    - df: pandas DataFrame containing time series data for multiple variables
    - target_col: string, name of the target variable column (e.g., 'WindSpeed10m')
    - feature_cols: list of strings, names of feature columns to use (including target if desired)
    - lag: number of past time steps to include as input features
    
    Returns:
    - X: 2D NumPy array of shape (samples, features * lag)
    - y: 1D NumPy array of target values (samples,)
    """
    data = df[feature_cols].values
    target_idx = feature_cols.index(target_col)
    
    X, y = [], []
    for i in range(lag, len(df)):
        # extract lagged observations for all features
        X.append(data[i-lag:i].flatten())  # flatten to 1D array of length features*lag
        y.append(data[i, target_idx])
    return np.array(X), np.array(y)


In [6]:
class RELM:
    """
    Regularized Extreme Learning Machine (RELM) as per Huang et al.
    Primal (N >= L):  beta = (I/C + H^T H)^(-1) H^T Y
    Dual   (N <  L):  beta = H^T (I/C + H H^T)^(-1) Y

    Supports activations: 'sigmoid' (stable), 'tanh', 'relu'
    """
    def __init__(self, n_hidden=100, activation='tanh', C=1.0, random_state=None):
        self.n_hidden = int(n_hidden)
        self.activation = activation
        self.C = float(C)
        self.random_state = random_state
        self.is_fitted = False

    def _init_weights(self, n_features):
        rng = np.random.default_rng(self.random_state)
        # Uniform draws as is common in ELM papers
        self.W = rng.uniform(-1, 1, size=(self.n_hidden, n_features))
        self.b = rng.uniform(-1, 1, size=(self.n_hidden,))

    def _activation(self, X):
        if self.activation == 'sigmoid':
            # numerically stable sigmoid
            X = np.clip(X, -500, 500)
            return 1.0 / (1.0 + np.exp(-X))
        if self.activation == 'tanh':
            return np.tanh(X)
        if self.activation == 'relu':
            return np.maximum(0.0, X)
        raise ValueError(f"Unknown activation: {self.activation}")

    def fit(self, X, y):
        X = np.asarray(X)
        y = np.asarray(y)
        if y.ndim == 1:
            y = y.reshape(-1, 1)

        N, d = X.shape
        self._init_weights(d)
        H = self._activation(X @ self.W.T + self.b)

        if N >= self.n_hidden:
            # Primal form
            A = (np.eye(self.n_hidden) / self.C) + (H.T @ H)
            B = H.T @ y
            self.beta = np.linalg.solve(A, B)
        else:
            # Dual form
            A = (np.eye(N) / self.C) + (H @ H.T)
            B = y
            self.beta = H.T @ np.linalg.solve(A, B)

        self.is_fitted = True
        return self

    def predict(self, X):
        if not self.is_fitted:
            raise RuntimeError("Model not fitted.")
        X = np.asarray(X)
        H = self._activation(X @ self.W.T + self.b)
        Y = H @ self.beta
        return Y.ravel() if Y.shape[1] == 1 else Y

    def rmse(self, X, y):
        yp = self.predict(X)
        return float(np.sqrt(mean_squared_error(y, yp)))

In [7]:
def create_multistep_targets(X, y, max_step=7):
    """
    Build (X_step, y_step) for horizons 1..max_step without recursion.
    For each step k, aligns X[:-k] with y[k:].
    """
    targets = []
    for step in range(1, max_step + 1):
        target = y[step:]
        X_valid = X[:-step]
        targets.append((X_valid, target))
    return targets

In [8]:
def safe_mape(y_true, y_pred, min_denom=1.0):
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    mask = np.abs(y_true) >= min_denom
    if np.sum(mask) == 0:
        return np.nan
    return np.mean(np.abs((y_true[mask] - y_pred[mask]) / y_true[mask])) * 100.0

def sde(y_true, y_pred):
    return float(np.std(np.array(y_true) - np.array(y_pred)))

In [9]:
class GreyWolfOptimizer:
    """
    Classic GWO (alpha, beta, delta) for continuous variables.
    Allows integer rounding for selected dims (e.g., n_hidden).
    """
    def __init__(self, obj_func, lb, ub, dim, num_wolves=20, max_iter=40,
                 integer_index=None, seed=2025):
        self.obj_func = obj_func
        self.lb = np.array(lb, dtype=float)
        self.ub = np.array(ub, dtype=float)
        self.dim = dim
        self.num_wolves = num_wolves
        self.max_iter = max_iter
        self.rng = np.random.default_rng(seed)
        self.integer_index = set(integer_index or [])

    def _clip(self, pos):
        return np.minimum(np.maximum(pos, self.lb), self.ub)

    def optimize(self):
        # Initialize wolves within bounds
        wolves = self.rng.uniform(self.lb, self.ub, (self.num_wolves, self.dim))

        def eval_pack(W):
            return np.array([self.obj_func(w) for w in W])

        fitness = eval_pack(wolves)

        # Sort and pick alpha, beta, delta
        def sort_wolves(w, f):
            idx = np.argsort(f)
            return w[idx], f[idx]

        wolves, fitness = sort_wolves(wolves, fitness)
        alpha, beta, delta = wolves[0].copy(), wolves[1].copy(), wolves[2].copy()
        alpha_score, beta_score, delta_score = float(fitness[0]), float(fitness[1]), float(fitness[2])

        history = [(0, alpha_score)]

        for t in range(1, self.max_iter + 1):
            a = 2.0 - 2.0 * (t / self.max_iter)  # linearly decreases 2 -> 0

            for i in range(self.num_wolves):
                X = wolves[i]

                def encircle(leader):
                    r1 = self.rng.random(self.dim)
                    r2 = self.rng.random(self.dim)
                    A = 2 * a * r1 - a
                    C = 2 * r2
                    D = np.abs(C * leader - X)
                    return leader - A * D

                X1 = encircle(alpha)
                X2 = encircle(beta)
                X3 = encircle(delta)

                newX = (X1 + X2 + X3) / 3.0
                newX = self._clip(newX)

                # enforce integer dims
                for idx in self.integer_index:
                    newX[idx] = np.round(newX[idx])

                wolves[i] = newX

            fitness = eval_pack(wolves)
            wolves, fitness = sort_wolves(wolves, fitness)

            if fitness[0] < alpha_score:
                alpha, alpha_score = wolves[0].copy(), float(fitness[0])
            beta, beta_score = wolves[1].copy(), float(fitness[1])
            delta, delta_score = wolves[2].copy(), float(fitness[2])

            history.append((t, alpha_score))

        return alpha, alpha_score, history

In [10]:
feature_columns = ['AirTemp','Azimuth','CloudOpacity','DewpointTemp','Dhi','Dni','Ebh',
                   'WindDirection10m','Ghi','RelativeHumidity','SurfacePressure','WindSpeed10m']
target_column = 'WindSpeed10m'
lag_steps = 12

In [11]:
USE_MULTISTEP_OBJECTIVE = False  # set True to optimize average RMSE over 1..H
MAX_STEP_OBJECTIVE = 7
RELM_ACTIVATION = 'tanh'
LOG10C_BOUNDS = (-3.0, 3.0)
N_HIDDEN_BOUNDS = (20, 600)

In [12]:
N_WOLVES = 20
MAX_ITERS = 60
SEED = 2025

In [13]:
X, y = create_multivariate_lagged_dataset(df, target_column, feature_columns, lag=lag_steps)

In [14]:
n_samples = X.shape[0]
train_end = int(0.7 * n_samples)
val_end = int(0.85 * n_samples)

X_train, y_train = X[:train_end], y[:train_end]
X_val,   y_val   = X[train_end:val_end], y[train_end:val_end]
X_test,  y_test  = X[val_end:], y[val_end:]

In [17]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled   = scaler.transform(X_val)
X_test_scaled  = scaler.transform(X_test)

In [18]:
def relm_val_rmse_one_step(params):
    """
    Objective: one-step-ahead validation RMSE.
    params: [log10(C), n_hidden]
    """
    logC, n_hidden = float(params[0]), int(np.round(params[1]))
    C = 10.0 ** logC
    n_hidden = max(1, n_hidden)

    model = RELM(n_hidden=n_hidden, activation=RELM_ACTIVATION, C=C, random_state=42)
    try:
        model.fit(X_train_scaled, y_train)
        rmse = model.rmse(X_val_scaled, y_val)
    except Exception:
        rmse = 1e9  # penalize numerical failures
    return rmse

def relm_val_rmse_multistep(params, max_step=7):
    """
    Objective: average RMSE over validation horizons 1..max_step (non-recursive).
    """
    logC, n_hidden = float(params[0]), int(np.round(params[1]))
    C = 10.0 ** logC
    n_hidden = max(1, n_hidden)

    model = RELM(n_hidden=n_hidden, activation=RELM_ACTIVATION, C=C, random_state=42)
    try:
        model.fit(X_train_scaled, y_train)

        # Build multistep targets **on the validation split**
        ms_val = create_multistep_targets(X_val_scaled, y_val, max_step=max_step)
        rmses = []
        for _, (Xv_step, yv_step) in enumerate(ms_val, 1):
            y_pred = model.predict(Xv_step)
            rmses.append(np.sqrt(mean_squared_error(yv_step, y_pred)))
        rmse = float(np.mean(rmses))
    except Exception:
        rmse = 1e9
    return rmse


In [19]:
if USE_MULTISTEP_OBJECTIVE:
    obj_func = lambda p: relm_val_rmse_multistep(p, max_step=MAX_STEP_OBJECTIVE)
else:
    obj_func = relm_val_rmse_one_step

In [20]:
lb = [LOG10C_BOUNDS[0], N_HIDDEN_BOUNDS[0]]
ub = [LOG10C_BOUNDS[1], N_HIDDEN_BOUNDS[1]]

In [21]:
gwo = GreyWolfOptimizer(
    obj_func=obj_func,
    lb=lb, ub=ub, dim=2,
    num_wolves=N_WOLVES, max_iter=MAX_ITERS,
    integer_index=[1],  # n_hidden is integer
    seed=SEED
)

In [23]:
best_params, best_val_score, history = gwo.optimize()
best_logC, best_hidden = float(best_params[0]), int(np.round(best_params[1]))
best_C = 10.0 ** best_logC

print(f"[GWO] Best validation score (RMSE): {best_val_score:.6f}")
print(f"[GWO] Best hyperparameters: C = {best_C:.6g}, n_hidden = {best_hidden}, activation = {RELM_ACTIVATION}")

[GWO] Best validation score (RMSE): 1.032335
[GWO] Best hyperparameters: C = 0.00140916, n_hidden = 581, activation = tanh


In [24]:
X_trainval_scaled = np.vstack([X_train_scaled, X_val_scaled])
y_trainval = np.concatenate([y_train, y_val])

final_model = RELM(n_hidden=best_hidden, activation=RELM_ACTIVATION, C=best_C, random_state=42)
final_model.fit(X_trainval_scaled, y_trainval)

In [25]:
test_rmse = final_model.rmse(X_test_scaled, y_test)
test_mae = mean_absolute_error(y_test, final_model.predict(X_test_scaled))
test_mape = safe_mape(y_test, final_model.predict(X_test_scaled))
test_sde  = sde(y_test, final_model.predict(X_test_scaled))

In [26]:
print("\n=== One-step Test Metrics ===")
print(f"MAE  : {test_mae:.6f}")
print(f"RMSE : {test_rmse:.6f}")
print(f"MAPE : {test_mape:.3f}%")
print(f"SDE  : {test_sde:.6f}")


=== One-step Test Metrics ===
MAE  : 0.785056
RMSE : 0.998352
MAPE : 30.234%
SDE  : 0.955556


In [27]:
MAX_STEP_EVAL = 7
multistep_data_test = create_multistep_targets(X_test_scaled, y_test, max_step=MAX_STEP_EVAL)

rows = []
for step, (step_X, step_y) in enumerate(multistep_data_test, 1):
    y_pred_step = final_model.predict(step_X)
    mae = mean_absolute_error(step_y, y_pred_step)
    rmse = np.sqrt(mean_squared_error(step_y, y_pred_step))
    mape_val = safe_mape(step_y, y_pred_step)
    sde_val = sde(step_y, y_pred_step)
    rows.append({"Step": step, "MAE": mae, "RMSE": rmse, "MAPE (%)": mape_val, "SDE": sde_val})

In [28]:
results_df = pd.DataFrame(rows, columns=["Step", "MAE", "RMSE", "MAPE (%)", "SDE"])
print("\n=== Multi-step Test Metrics (non-recursive, 1..7) ===")
print(results_df.to_string(index=False))


=== Multi-step Test Metrics (non-recursive, 1..7) ===
 Step      MAE     RMSE  MAPE (%)      SDE
    1 0.852299 1.080472 32.731733 1.041054
    2 0.937051 1.190286 35.350458 1.154622
    3 1.021617 1.301549 38.182487 1.269037
    4 1.101322 1.401289 40.719982 1.371169
    5 1.169479 1.481541 43.041351 1.453110
    6 1.225065 1.540986 45.021263 1.513702
    7 1.268643 1.583406 46.783640 1.556896
